### Install the packages

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


In [3]:
# store the url
url = "https://mtu.libanswers.com/"
resp = requests.get(url)
content = resp.text
soup = BeautifulSoup(content, 'html.parser')

### Extract content

In [4]:
# find all faqs

# store faqs data
all_faqs = []
faq_links = []
faq_topics = []
faq_last_update = []
faq_views = []


# selects from "all" tab
all_tab_container = soup.find('div', id='s-la-box-65168-container-tab1')

# Find all faqs in the 'all' tab
faq_items = all_tab_container.find_all('li', class_='s-la-faq-listing')

# Iterate through each FAQ item and print its contents
for faq_item in faq_items:
    # Extract the FAQ question and its associated link
    faq_question = faq_item.find('div', class_='s-la-faq-listing-q').a.text.strip()
    faq_link = faq_item.find('div', class_='s-la-faq-listing-q').a['href']
    # extract number of views
    views_label_span = faq_item.find('span', class_='metalabel', string='Views:')
    views_number = views_label_span.find_next('span', class_='metavalue').text.strip()
    # Extract the topics for the current FAQ
    topics_list = faq_item.find('ul', class_='list-inline metavalue')
    if topics_list:
        topics = [topic.text.strip() for topic in topics_list.find_all('a')]
    else:
        topics = "NaN"
    # last updated
    # Extract the last updated information for the current FAQ
    last_updated_span = faq_item.find('span', class_='metalabel', string='Last Updated:')
    last_updated_info = last_updated_span.find_next('span', class_='metavalue').text.strip()
    # store the data 
    all_faqs.append(faq_question)
    faq_links.append(faq_link)
    faq_views.append(views_number)
    faq_topics.append(topics)
    faq_last_update.append(last_updated_info)

# extract featured faqs
featured_tab_container = soup.find('div', id='s-la-box-65168-container-tab0')
# find all featured faqs
faq_items = featured_tab_container.find_all('li', class_='s-la-faq-listing')
featured_faq = []

# get the faq text
for faq_item in faq_items:
    # Extract the FAQ question and its associated link
    faq_question = faq_item.find('div', class_='s-la-faq-listing-q').a.text.strip()
    featured_faq.append(faq_question)
    
# create new list that determines if an faq is featured or not.
featured = []
for i in all_faqs:
    if i in featured_faq:
        featured.append('Yes')
    else:
        featured.append('No')
        
# print (len(featured),len(all_faqs) )




### Create DataFrame


In [14]:
# df = pd.DataFrame(np.column_stack([all_faqs, faq_links, faq_views,faq_topics,faq_last_update, featured]),
#                  columns=['Faqs', 'Faq Links', 'Views', 'Topics', 'Last Update', 'Featured'])
df = pd.DataFrame(
    {'Faqs': all_faqs,
     'Faq_Links': faq_links,
     'Views': faq_views,
     'Topics': faq_topics,
     'Last_Update': faq_last_update,
     'Featured': featured
    })
# df.head(3)
# export df
df.to_csv('faq.csv', sep='\t', encoding='utf-8', index= False)